In [1]:
import qiskit
import qiskit_ibm_runtime
print(f"Qiskit version: {qiskit.__version__}")
print(f"Runtime version: {qiskit_ibm_runtime.__version__}")

Qiskit version: 2.1.0
Runtime version: 0.40.1


## 8교시 : Error mitigation and suppression techniques

마지막 실습에서는 Qiskit Runtime에서 이용 가능한 오차 억제와 오류 완화 테크닉에 대해 알아보도록 하겠습니다.

먼저 아래의 셀을 완성하여 Qiskit 런타임 서비스에 접속해 사용할 백엔드를 설정해보세요. (이번에는 실제로 양자 컴퓨터를 사용하지는 않습니다.)

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator

service = #
backend = #

### 8-1 : Dynamical decoupling

양자 컴퓨터에서 발생하는 오류를 억제하는 한 가지 방법은 큐비트에 발생하는 오류가 상쇄될 수 있도록 큐비트를 뒤집어주는 것입니다. 이러한 방법을 dynamical decoupling (DD) 라고 부릅니다.

![Depiction of dynamical decoupling](img/dd.avif)

위의 그림은 양자 회로 연산 중 X 게이트를 가해 dynamical decoupling을 수행하는 것을 그림으로 나타낸 것입니다. 왼쪽의 가상의 회로는 실제 양자 컴퓨터에서 오른쪽의 물리적인 전자기 펄스로 변환되는데, 아래와 같이 첫 번째 큐비트가 가만히 있는 동안에 X 펄스를 두 번 가함으로써 이 큐비트에 들어오는 노이즈를 상쇄시킬 수 있습니다.

Dynamical decoupling은 회로를 실행하는 중에 게이트가 가해지지 않고 가만히 있는 큐비트에만 가하는 것이 좋다는 사실을 참고하세요. Dynamical decoupling은 큐비트에 들어오는 노이즈를 상쇄시키는 역할을 하지만 양자 게이트를 가할 때에도 오류가 발생할 수 있습니다. 따라서 dynamical decoupling을 너무 많이 사용하면 억제하고자 하는 노이즈보다 이로 인해 발생하는 노이즈가 더 커질 수도 있습니다.

Dynamical decoupling은 [dynamical decoupling options](https://quantum.cloud.ibm.com/docs/api/qiskit-ibm-runtime/options-dynamical-decoupling-options)의 `enable` 설정을 `True` 로 둠으로써 활성화할 수 있습니다. `sequence_type` 옵션을 설정하여 dynamical decoupling을 수행하는 펄스 시퀀스를 선택할 수 있습니다. (기본값은 `"XX"` 입니다.)

아래의 코드는 Estimator에서 dynamical decoupling을 활성화하고 시퀀스를 선택하는 예시입니다.

In [ ]:
estimator = Estimator(mode=backend)
estimator.options.dynamical_decoupling.enable = True
estimator.options.dynamical_decoupling.sequence_type = "XpXm"

### 8-2 : Pauli twirling

양자 회로의 오류를 억제하는 다른 방법으로는 Pauli twirling이 있습니다. 이 테크닉 자체가 노이즈를 억제하는 효과는 크지 않지만, twirling을 가함으로써 복잡하게 발생할 수 있는 노이즈의 특성을 단순화하고 이후 오차 완화 기술을 더 효과적으로 적용할 수 있습니다. 

![Depiction of Pauli twirling](img/pauli_twirling.avif)

Pauli twirling은 수행하고자 하는 양자 게이트 앞뒤에 게이트의 연산을 방해하지 않는 랜덤한 파울리 게이트를 가하는 방식으로 구현됩니다. 즉, 하나의 똑같은 회로만을 반복하는 것이 아니라 그 회로와 동등한 여러 가지 회로를 랜덤하게 실행하는 것입니다.

현대의 양자컴퓨터에서 발생하는 오류는 주로 2-큐빗 게이트에서 발생하기 때문에 이 테크닉은 보통 2-큐빗 게이트에만 적용이 되곤 합니다. 아래 그림은 Pauli CNOT 게이트와 ECR 게이트를 수행하는 몇 가지 방법의 예시를 나타냅니다.

![Depiction of gate twirls](img/gate_twirls.avif)

Pauli twirling은 [twirling options](https://quantum.cloud.ibm.com/docs/api/qiskit-ibm-runtime/options-twirling-options)의 `enable_gates` 옵션을 `True` 로 설정함으로써 활성화할 수 있습니다. 그 외 주요 설정으로는:

*   `num_randomizations`: Twirling을 적용해 랜덤하게 생성할 (서로 동등한) 회로의 종류 갯수입니다.
*   `shots_per_randomization`: 랜덤하게 선택된 각 회로마다 실행할 shot의 개수입니다.

아래의 코드는 Estimator에서 Pauli twirling을 활성화하고 옵션을 설정하는 예시입니다. 옵션을 따로 설정하지 않아도 기본값으로 실행이 가능합니다.

In [ ]:
estimator = Estimator(mode=backend)
estimator.options.twirling.enable_gates = True
estimator.options.twirling.num_randomizations = 32
estimator.options.twirling.shots_per_randomization = 100

### 8-3 : Twirled readout error extinction (TREX)

[Twirled readout error extinction (TREX)](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.105.032620)은 기댓값의 측정 오류를 완화할 수 있는 테크닉입니다. 이것은 측정 연산을 수행할 때에 랜덤하게 (1) X 게이트, (2) 측정, (3) 고전적인 NOT 게이트를 적용하는 방법으로 0과 1을 뒤집어 측정하는 방식으로 구현됩니다.

![Depiction of measurement twirling](img/measurement_twirling.avif)

측정 오류가 있을 때에, 측정 연산에 twirling을 가하는 것은 측정 오류 행렬을 대각화하는 효과가 있으며, 이 경우 측정 오류를 역으로 계산하여 원래의 값을 쉽게 찾을 수 있게 됩니다. 측정 오류 행렬은 회로를 실행하기 전에 간단한 샘플링을 추가하는 것으로 얻을 수 있습니다.

TREX는 Estimator의 [Qiskit Runtime resilience options](https://quantum.cloud.ibm.com//docs/api/qiskit-ibm-runtime/options-resilience-options-v2)의 `measure_mitigation` 옵션을 `True`로 설정하는 것으로 활성화할 수 있습니다. 측정 오류 학습에 관여하는 옵션은 [이 API 문서](https://quantum.cloud.ibm.com/docs/api/qiskit-ibm-runtime/options-measure-noise-learning-options)에서 확인하실 수 있습니다. 앞의 gate twirling과 같이, 랜덤하게 실행할 회로의 종류와 측정 횟수를 직접 설정할 수 있습니다.

In [ ]:
estimator = Estimator(mode=backend)
estimator.options.resilience.measure_mitigation = True
estimator.options.resilience.measure_noise_learning.num_randomizations = 32
estimator.options.resilience.measure_noise_learning.shots_per_randomization = 100

<span id="zne" />

### 8-4 : Zero-noise extrapolation (ZNE)

Zero-noise extrapolation (ZNE, 무잡음 추정법) 은 관측가능량의 기댓값을 측정할 때에 오차를 완화하기 위해 사용할 수 있는 테크닉입니다. 이 방법은 실제로 꽤 효과적이지만 추정값의 정확도를 보장해주지는 않습니다.

ZNE는 두 단계로 이루어집니다:

1.  *노이즈 증폭*: 양자 회로의 노이즈를 임의로 증폭하여 결과를 샘플링합니다.
2.  *추정*: 노이즈를 증폭하여 얻은 결과로부터 노이즈가 없을 때의 이상적인 결과를 추정합니다.

![Depiction of ZNE](img/zne.avif)

노이즈의 증폭과 추정 단계 모두 여러 가지 방법을 적용하여 수행될 수 있습니다. Qiskit Runtime에서는 기본값으로 "digital gate folding," 방법을 사용해 노이즈를 증폭하며, 추정 단계에서는 선형 추정법과 지수 추정법을 적용한 결과를 각각 보여줍니다. 이 중, 게이트 폴딩(국소적 접기) 방법은 2-큐빗 게이트 연산과 역연산을 반복하여 노이즈를 증폭시키게 됩니다.

ZNE는 Estimator의 [Qiskit Runtime resilience options](https://quantum.cloud.ibm.com/docs/api/qiskit-ibm-runtime/options-resilience-options-v2)의 `zne_mitigation` 옵션을 `True`로 설정하여 활성화할 수 있습니다. ZNE에 적용되는 다양한 옵션들은 [이 API 문서](https://quantum.cloud.ibm.com/docs/api/qiskit-ibm-runtime/options-zne-options)에서 확인할 수 있습니다. 특히 아래의 옵션은 같이 확인해봅시다:

*   `noise_factors`: 노이즈를 증폭하는 배율을 지정합니다.
*   `extrapolator`: 추정에 사용할 함수의 형태를 지정합니다.


In [ ]:
estimator = Estimator(mode=backend)
estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.noise_factors = (1, 3, 5)
estimator.options.resilience.zne.extrapolator = "exponential"

Q. 선형 추정법을 적용하여 zne를 수행하려면 어떻게 하면 될까요?

In [ ]:
# 

### 8-5 : 오류 완화의 간편한 설정

위에서 언급한 오류 완화 테크닉들은 Estimator의 `resilience_level`을 설정하는 것으로 간편하게 설정할 수 있습니다. 이 값을 높게 설정할 수록 더 많은 테크닉을 적용할 수 있지만, 오류 완화 테크닉을 많이 적용할 수록 더 많은 계산과 샘플링이 필요해진다는 점도 고려해야 합니다.

아래의 표는 Estimator에서 선택할 수 있는 resilience level과 각 레벨에서 적용되는 오류 완화 테크닉을 나타냅니다. Sampler는 현재 resilience level 옵션을 지원하지 않습니다.

| Resilience Level | Definition                                                                                            | Technique                                                     |
| ---------------- | ----------------------------------------------------------------------------------------------------- | ------------------------------------------------------------- |
| 0                | No mitigation                                                                                         | None                                                          |
| 1 \[Default]     | Minimal mitigation costs: Mitigate error associated with readout errors                               | Twirled Readout Error eXtinction  (TREX) measurement twirling |
| 2                | Medium mitigation costs. Typically reduces bias in estimators, but is not guaranteed to be zero-bias. | Level 1 + Zero Noise Extrapolation (ZNE) and gate twirling    |


In [ ]:
# Primitive를 설정할 때에 옵션으로 resilience_level을 설정할 수 있습니다.
estimator = Estimator(backend, options={"resilience_level": 2})